In [21]:
import numpy as np
import matplotlib.pyplot as plt
from torch import nn
from torchvision import models
import torch
import gym, sys
import cv2
from tqdm import tqdm
from IPython.display import clear_output
from collections import Counter
from array2gif import write_gif
def data_transform(frame) :
    # breakout-v4
    # [width, height, channels] (기존)
    # [channels, height, width] (변경)
    frame = np.array(frame).transpose(2,1,0)
    frame = np.expand_dims(frame, 0) 
    assert np.sum(np.isnan(frame)) == 0, "state error!"
    return frame.astype(np.float32)
    
from rlxai.gym_wrapper import FireResetEnv , WarpFrame , ScaledFloatFrame , FrameStack


In [22]:
stack_n = 4
# env = gym.make("Breakout-v4") # Deterministic
env = gym.make("BreakoutDeterministic-v4") # 
# env = gym.make("BreakoutNoFrameskip-v4") # 
if 'FIRE' in env.unwrapped.get_action_meanings():
    env = FireResetEnv(env)
env = WarpFrame(env)
env = ScaledFloatFrame(env)
env = FrameStack(env,k=stack_n )

random_seed = 54852  # set random seed if required (0 = no random seed)
action_dim = env.action_space.n
device = "cpu"
from rlxai.agent.agent_selection import get_agent
model_name = "PPO"
save_path = "./../model"
save_path = "/home/docker/Project/XAI-RL/src/rlxai/distributed/model"
model,_,_ = get_agent(model_name=model_name,save_path=save_path,device=device,action_dim=action_dim,state_dim=stack_n ,load_model=True)

load


In [23]:
if random_seed:
    print("--------------------------------------------------------------------------------------------")
    print("setting random seed to ", random_seed)
    torch.manual_seed(random_seed)
    env.seed(random_seed)
    np.random.seed(random_seed)

--------------------------------------------------------------------------------------------
setting random seed to  54852


In [24]:
ACTION_MEANING = env.unwrapped.get_action_meanings()

In [25]:
s = env.reset()
img_collection = [env.render(mode="rgb_array").transpose(1,0,2)]
iter = 0
ACTION_LIST = []
R = 0
info = {'lives' : 5}
last_lives = info['lives']
while True:
    action_dict = model.select_action(s,False)
    if info["lives"] < last_lives :
        action = 1
        last_lives = info["lives"]
    else :
        action = int(action_dict["action"])
    # action = int(action_dict["action"])
    s, r, done, info = env.step(action)
    R += r
    ACTION_LIST.append(ACTION_MEANING[action])
    #############################################
    img_collection.append(env.render(mode="rgb_array").transpose(1,0,2))
    iter += 1
    if done:
        break
# print(", ".join(ACTION_LIST))
print(f"Reward : {R}")
print(Counter(ACTION_LIST))
if R < 30 :
    print(f"Reward : {R} is very small so we don't make gif file")
else :
    file_path = "./breakout.gif"
    write_gif(img_collection, file_path)
    print(f"Reward : {R} save gif...")

Reward : 60.0
Counter({'LEFT': 518, 'RIGHT': 496, 'FIRE': 71, 'NOOP': 62})
Reward : 60.0 save gif...


In [12]:
# write_gif([img for img in img_collection[0:20]], file_path)